# AUTOGRAD: AUTOMATIC DIFFERENTIATION / 자동 미분

> The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

> autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다. 이는 실행-기반-정의(define-by-run) 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라집니다.



Create a tensor and set requires_grad=True to track computation with it / tensor를 생성하고 requires_grad=True 를 설정하여 연산을 기록합니다.

In [2]:
import torch                                     # Always import "torch" package first

x = torch.ones(2, 2, requires_grad=True)         # requires_grad는 
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
